# Wordle Analysis
--------
**What is Wordle?**

Wordle is a web-based word game developed by Welsh-born software engineer Josh Wardle. Players have six attempts to guess a five-letter word

----

<img src="images/wordle_splash.png" width="400">

In [ ]:
import string
import time
import matplotlib.pyplot as plt
import numpy as np
import solve_wordle

# Getting the dictionary

**How did we get the words?**

1. Right-click and inspect
2. Go to sources and download the javascript
3. Prettify the source and find the word lists
4. Save the words to JSON

------
<img src="images/how_to_get_words.png" width="800">

------
<img src="images/words_json.png" width="600">

# Load the dictionary

In [ ]:
all_words = solve_wordle.load_words("wordle_words.json")
print(f"Total number of words: {len(all_words)}")

In [ ]:
print("First 100 words:")
print(all_words[0:100])

# Analysis of words and letters

In [ ]:
words_by_letter = solve_wordle.get_words_by_letter(all_words)
word_count_by_letter = {letter: len(words_by_letter[letter]) for letter in words_by_letter}
# for letter in word_count_by_letter:
#     print(letter, word_count_by_letter[letter])

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
ax.bar(range(26), word_count_by_letter.values(), tick_label=list(word_count_by_letter.keys()));
ax.set_title("Number of words for each letter", loc='left');

# Which words has the most common letters?

```Python
for word in all_words:
    words_score = 0
    for letter in word:
        words_score += len(words_by_letter[letter])
```

In [ ]:
guesses = solve_wordle.get_most_unique_score_look_at_letter_score(all_words, all_words, [], [])
for score, word in guesses[0:10]:
    print(f"{word} with a score of {int(score)}")


# Which word overlaps with the most other words?

This eliminates double counting words that show up for multiple letters. 

```Python
for word in all_words:
    words_that_overlap = set()
    for letter in word:
        words_that_overlap.union(words_by_letter[letter])
    score = len(words_that_overlap)
```

In [ ]:
guesses = solve_wordle.get_most_unique_score_look_at_allowed_words(all_words, all_words, [], [])
for score, word in guesses[0:10]:
    print(f"{word} with a score of {score}")

# First word

Note this was from February 4, 2022.

Plug in **TOEAS** and figure out what info Wordle gives us. Remember that 
- green = the letter is in the word and in the correct spot
- yellow = the letter is in the word but in the wrong spot
- gray = the letter is not in the word

<img src="images/toeas.png" width="400">



# What did we learn?
Here is the info we learned:
- The word has **T** but it is not the first letter
- The word does not have an **O**
- The word has at least one **E** in the 3rd spot
- The word has at least one **A** in the 4th spot
- The word does not have an **S**

Now translate that into data.

In [ ]:
required_letters = set()
exclude_letters = set()

required_letters.add(('t', None)) # The word has T but it is not the first letter
exclude_letters.add(('t', 0))     # The word has T but it is not the first letter

exclude_letters.add(('o', None))  # The word does not have an O

required_letters.add(('e', 2))    # The word has at least one E in the 3rd spot

required_letters.add(('a', 3))    # The word has at least one A in the 4th spot

exclude_letters.add(('s', None))  # The word does not have an S

# Find all the words that are allowed
Use the data to filter the dictionary to only words that satisfy the rules we created

In [ ]:
allowed_words, not_allowed_words = solve_wordle.get_words_with_letters(all_words, required_letters, exclude_letters)
print(f"Number of words eliminated {len(not_allowed_words)}")
print(f"Number of potential words left {len(allowed_words)}")
for i, word in enumerate(allowed_words):
    print(f"{i}. {word}")

# Find the guess for the second word
Now that we have an algorithm we just repeat until we find the solution

In [ ]:
guesses = solve_wordle.get_most_unique_score_look_at_allowed_words(allowed_words, all_words, [], [])
for score, word in guesses:
    print(f"{word} with a score of {int(score)}")

# Second word

<img src="images/cleat.png" width="400">

In [ ]:
exclude_letters.add(('c', None))  # The word does not have an C
required_letters.add(('l',1)) # The word has at least one L in the 2rd spot
required_letters.add(('t', 4))    # The word has at least one T in the 5th spot

In [ ]:
allowed_words, not_allowed_words = solve_wordle.get_words_with_letters(all_words, required_letters, exclude_letters)
print(f"Number of words eliminated {len(not_allowed_words)}")
print(f"Number of potential words left {len(allowed_words)}")
for i in allowed_words:
    print(i)

# Find the guess for the thrid word

In [ ]:
guesses = solve_wordle.get_most_unique_score_look_at_all_words(allowed_words, allowed_words, [], [])
for score, word in guesses:
    print(f"{word} with a score of {int(score)}")

# Third word

<img src="images/pleat.png" width="400">

# Compare alogirhtms

Let's look at how many guesses it takes to find 1,000 words in the dictionary and answer the following questions.

- How many guesses does it take to find a word?
- How long does the computer take to calculate the answer?

In [ ]:
x = range(0, 15)
def plot_bar(number_words_by_count):
    y = [number_words_by_count.get(i,0) for i in x]
    
    fig, ax = plt.subplots(figsize=(16,4))
    ax.bar(x, y_allowed_letters, tick_label=x);
    ax.set_xlim([0.5, 14.5])
    ax.set_ylim([0, 400])
    ax.set_xlabel('Number guesses to solve')
    ax.set_title("Number of words", loc='left');
    return y

# Just look at letter scores for allowed words

In [ ]:
dt_letter_score = 7.276570796966553
number_words_by_count = {2: 12, 3: 151, 4: 332, 5: 269, 6: 127, 7: 51, 8: 28, 9: 19, 10: 6, 11: 2, 12: 1, 13: 2}
y_allowed_letters = plot_bar(number_words_by_count)

# Just look at letter scores for all words
Give a small bonus to allowed words

In [ ]:
dt_letter_score = 14.363650798797607
number_words_by_count = {2: 9, 3: 138, 4: 339, 5: 281, 6: 134, 7: 48, 8: 21, 9: 18, 10: 7, 11: 2, 12: 1, 13: 2}
y_all_letters = plot_bar(number_words_by_count)

# Look at allowed words

In [ ]:
dt_allowed_words_score = 4591.0739669799805
number_words_by_count = {2: 15, 3: 144, 4: 345, 5: 260, 6: 123, 7: 55, 8: 26, 9: 18, 10: 7, 11: 5, 14: 2}
y_allowed_words = plot_bar(number_words_by_count)

# Look at all words to find best letters

In [ ]:
dt = 5000.753873825073
number_words_by_count = {2: 2, 3: 99, 4: 370, 5: 365, 6: 143, 7: 17, 8: 4}
y_all_words = plot_bar(number_words_by_count)

# Compare all algorithms

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
ax.step(x, y_allowed_letters, where='mid', label='Letter score allowed words');
ax.step(x, y_all_letters, where='mid', label='Letter score all words');
ax.step(x, y_allowed_words, where='mid', label='Word set allowed words');
ax.step(x, y_all_words, where='mid', label='Word set all words');
ax.set_xlim([0.5, 14.5])
ax.set_ylim([0, 400])
ax.set_xlabel('Number guesses to solve')
ax.set_xticks(x)
ax.set_xticklabels(x)
ax.legend(loc='best')
ax.set_title("Number of words", loc='left');

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
ax.barh(range(4), 
        [sum(y_allowed_letters[7:]), sum(y_all_letters[7:]), sum(y_allowed_words[7:]), sum(y_all_words[7:])], 
        tick_label=['Letter score allowed words', 'Letter score all words', 'Word set allowed words', 'Word set all words'])
ax.invert_yaxis()
ax.set_xlabel('Number words requiring more than 6 guesses')
ax.set_title("Wordle fails by algorithm", loc='left');

# Other ideas

## What about paying attention to letter placement

In [ ]:
letter_count_by_location = np.zeros([5, 26])
letter_to_index = {letter: i for i, letter in enumerate(string.ascii_lowercase)}
for word in all_words:
    for i in range(5):
        letter_count_by_location[i, letter_to_index[word[i]]] += 1
fig, ax = plt.subplots(figsize=(17,6))
im = ax.imshow(letter_count_by_location)
ax.set_xticks(np.arange(26))
ax.set_xticklabels(string.ascii_lowercase)
cbar = ax.figure.colorbar(im, ax=ax, orientation='horizontal')
ax.set_title("Count of letters by location in word", loc='left');